# Introduction to PyTerrier

_IN4325: Information retrieval lecture, TU Delft_

**Part 4: Evaluation & experiments**

This part focuses on running experiments and evaluating retrieval and ranking models using PyTerrier. We'll learn how to use query relevances (QRels) in order to compute ranking metrics and perform statistical significance testing.


In [1]:
pip install python-terrier==0.10.0

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pyterrier as pt

if not pt.started():
    pt.init(tqdm="notebook")

PyTerrier 0.10.0 has loaded Terrier 5.9 (built by craigm on 2024-05-02 17:40) and terrier-helper 0.0.8

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


## Queries and query relevance

Let's start by loading an existing dataset. We'll pick one from [the list](https://pyterrier.readthedocs.io/en/latest/datasets.html#available-datasets) that has topics (queries) and QRels available, such as `vaswani`:


In [3]:
dataset = pt.get_dataset("vaswani")
index = dataset.get_index(variant="terrier_stemmed")

Let's first have a look at the queries (topics) in this dataset. Note that this dataset only has one set of queries and QRels (you can see this in the table). If a dataset has multiple sets, you'll need to specify one, for example: `dataset.get_topics("test")`.


In [4]:
dataset.get_topics()

,qid,query
0,1,measurement of dielectric constant of liquids ...
1,2,mathematical analysis and design details of wa...
2,3,use of digital computers in the design of band...
3,4,systems of data coding for information transfer
4,5,use of programs in engineering testing of comp...
...,...,...
88,89,tunnel diode construction and its electrical c...
89,90,electronic density of states at the surface of...
90,91,resistivity of metallic thin films related to ...
91,92,the phenomenon of radiation caused by charged ...


Along with the set of queries, we can inspect the corresponding relevance data (QRels). These are tuples of (query, document, label). Note that labels can be either binary (`0` or `1`) or graded (for example, `0` to `3`, where a higher number encodes higher relevance). Tuples where the label is `0` (i.e., no relevance) are often omitted from the QRels.


In [5]:
dataset.get_qrels()

,qid,docno,label
0,1,1239,1
1,1,1502,1
2,1,4462,1
3,1,4569,1
4,1,5472,1
...,...,...,...
2078,93,9875,1
2079,93,9956,1
2080,93,10497,1
2081,93,11191,1


## Experiments

Using the QRels and the output of a ranking model (i.e., a _ranking_ or a _run_), we can compute ranking metrics that evaluate the quality of our results. Let's start with a BM25 model:


In [6]:
bm25 = pt.BatchRetrieve(index, wmodel="BM25")

An _experiment_ in PyTerrier automates the retrieval/ranking and evaluation process. This means that you can get a comprehensive evaluation of models or systems on some test set in a single line of code:


In [7]:
from pyterrier.measures import RR, nDCG, MAP

pt.Experiment(
    [bm25],
    dataset.get_topics(),
    dataset.get_qrels(),
    eval_metrics=[RR @ 10, nDCG @ 20, MAP],
)

,name,RR@10,nDCG@20,AP
0,BR(BM25),0.719875,0.415377,0.296517


Here, we have evaluted our BM25 retriever on the dataset we loaded earlier. We used three of the most common IR metrics, but [PyTerrier supports a very large number of additional metrics](https://pyterrier.readthedocs.io/en/latest/experiments.html#evaluation-measures-objects) through its integration of the [ir-measures](https://github.com/terrierteam/ir_measures) library.

**Important**: Usually, a value of `0` indicates _no relevance_, while a value greater than `0` indicates _relevance_. However, this is not always the case; a popular example is the TREC Deep Learning passage ranking task, where [only values greater than `1` indicate relevance](https://trec.nist.gov/data/deep2020.html). In those cases, the threshold can by adjusted by constructing the metric as, for example, `RR(rel=2)`.

Note that we used the `@` operator to limit the retrieval depth (number of documents per query) for specific metrics. Furthermore, since both queries (topics) and QRels are simply packaged in a `pandas.DataFrame`, you can easily provide your own data, as long as it is in the right format.

### Performance comparisons

Experiments let you easily compare multiple ranking approaches. Let's load another ranker, which uses standard TF-IDF, to see how much BM25 improves over it:


In [8]:
tf_idf = pt.BatchRetrieve(index, wmodel="TF_IDF")

We can now run an experiment that evaluates both models on the same data. For better readability, we assign custom names to both approaches.

Comparing the approaches in the same experiment allows us to automatically have _statistical significance testing_ performed. By setting `baseline=0`, we tell the function to compute the $p$-values with respect to the first approach (TF-IDF). Furthermore, [PyTerrier supports a number of correction methods](https://pyterrier.readthedocs.io/en/latest/experiments.html#significance-testing); here, we apply Bonferroni correction:


In [9]:
pt.Experiment(
    [tf_idf, bm25],
    dataset.get_topics(),
    dataset.get_qrels(),
    names=["TF-IDF", "BM25"],
    eval_metrics=[RR @ 10, nDCG @ 20, MAP],
    baseline=0,
    correction="bonferroni",
)

,name,AP,nDCG@20,RR@10,AP +,AP -,AP p-value,AP reject,AP p-value corrected,nDCG@20 +,nDCG@20 -,nDCG@20 p-value,nDCG@20 reject,nDCG@20 p-value corrected,RR@10 +,RR@10 -,RR@10 p-value,RR@10 reject,RR@10 p-value corrected
0,TF-IDF,0.290905,0.410850,0.694666,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,False,NaN
1,BM25,0.296517,0.415377,0.719875,46.0,45.0,0.237317,False,0.474634,31.0,34.0,0.309461,False,0.618923,9.0,2.0,0.034943,False,0.069886


### Saving results

Experiments allow for saving the obtained results (runs) in the common TREC format:


In [10]:
from pathlib import Path

results_dir = Path("results")
results_dir.mkdir(exist_ok=True)

pt.Experiment(
    [tf_idf, bm25],
    dataset.get_topics(),
    dataset.get_qrels(),
    names=["TF-IDF", "BM25"],
    eval_metrics=[RR @ 10, nDCG @ 20, MAP],
    save_dir=str(results_dir),
)

,name,RR@10,nDCG@20,AP
0,TF-IDF,0.694666,0.410850,0.290905
1,BM25,0.719875,0.415377,0.296517


This is useful when results need to be shared or processed futher. In addition, PyTerrier will re-use saved results rather than re-computing them. This behavior can be disabled by setting `save_mode="overwrite"`.

## Further reading

Check out the [section on experiments](https://pyterrier.readthedocs.io/en/latest/experiments.html) in the documentation.
